In [1]:
HUGGINGFACE_TOKEN = "" # ATTENTION: to get a token: https://huggingface.co/settings/tokens

To set up a Retrieval-Augmented Generation (RAG) system using your Mac with the Ollama platform, we’ll need to handle a few main components:

1.	LLM Model (using Ollama): We’ll call an open-source language model through Ollama for generation.
2.	Data Storage and Retrieval: This component indexes and retrieves relevant documents in response to a query.
3.	Application Logic: This ties together the retrieval and generation, handling inputs and outputs.

# 1. LLM Models (using Ollama)
- Download and install ollama from ollama.com
- Install some LLM models in your machine:
  ```ollama install <model_name>```
- List the existing models
- Consider a Model with Retrieval-Augmented Fine-Tuning (like LlamaIndex, GPT-Neo)

In [2]:
!ollama list

NAME                        ID              SIZE      MODIFIED     
nomic-embed-text:latest     0a109f422b47    274 MB    7 days ago      
llama2-uncensored:latest    44040b922233    3.8 GB    8 days ago      
deepseek-r1:1.5b            a42b25d8c10a    1.1 GB    2 weeks ago     
llama3.2:latest             a80c4f17acd5    2.0 GB    4 weeks ago     
llava:latest                8dd30f6b0cb1    4.7 GB    7 months ago    
llama3:latest               a6990ed6be41    4.7 GB    9 months ago    


In [3]:
# !pip install ollama 

def ollama(model, system_prompt, user_prompt):
    import ollama  # https://pypi.org/project/ollama/

    try:
        response = ollama.chat(model=model,
                               messages=[{'role': 'system', 'content': system_prompt},
                                         {'role': 'user', 'content': user_prompt}]
        )
        return response['message']['content']

    except ollama.ResponseError as e:
        print('Error:', e.error)
        if e.status_code == 404:
            ollama.pull(model)
            print("Re-run this and it will work! We pulled the model for you!") 
            return None

Testing call_ollama() function

In [4]:
model = "llama3.2"
system_prompt = "You are a helpful assistant."
user_prompt = "Say hello world as an emoji!"

response = ollama(model, system_prompt, user_prompt)

In [5]:
response

'🌎'

## Web Scraper into markdown

In [6]:
# !pip install crawl4ai
# !crawl4ai-setup

import asyncio
import nest_asyncio
nest_asyncio.apply()

from crawl4ai import AsyncWebCrawler, BrowserConfig, CrawlerRunConfig, CacheMode

async def url2md(url):
    browser_conf = BrowserConfig(headless=True)  # Run in headless mode
    run_conf = CrawlerRunConfig(cache_mode=CacheMode.BYPASS)

    async with AsyncWebCrawler(config=browser_conf) as crawler:
        result = await crawler.arun(url=url, config=run_conf)

        if result.success:
            return result.markdown_v2.raw_markdown  # Return extracted content
        else:
            return f"Error: {result.error_message}"  # Handle errors gracefully

In [7]:
def save_markdown(file_path, content):
    try:
        with open(file_path, "w", encoding="utf-8") as file:
            file.write(content)
        return file_path
    except Exception as e:
        print(f"Error saving file: {e}")

Example:

In [8]:
url = 'https://northwave-cybersecurity.com'
markdown_website = asyncio.run(url2md(url))

[INIT].... → Crawl4AI 0.4.248
[FETCH]... ↓ https://northwave-cybersecurity.com... | Status: True | Time: 0.99s
[SCRAPE].. ◆ Processed https://northwave-cybersecurity.com... | Time: 33ms
[COMPLETE] ● https://northwave-cybersecurity.com... | Status: True | Total: 1.03s


In [9]:
len(markdown_website)

17307

In [10]:
file_path = 'northwave_site.md'
content = markdown_website
save_markdown(file_path, content)

'northwave_site.md'

# 2. Loading .pdf and .docx to a Data Storage (using FAISS IndexFlatL2)

### Converting from .pdf to text

In [11]:
# !pip install pymupdf
import fitz  # PyMuPDF for PDFs

def pdf2txt(file_path):
    text = ""
    with fitz.open(file_path) as doc:
        for page in doc:
            text += page.get_text("text")
    return text

Example:

In [12]:
file_path = 'Masters_Graduation_Project_Plan___Tudor_Dragan.pdf'
text_from_pdf = pdf2txt(file_path)
len(text_from_pdf)

22614

### Converting from .docx to text (using python-docx)

In [13]:
# !pip install python-docx
import docx

def docx2txt(file_path):
    doc = docx.Document(file_path)
    text = "\n".join([paragraph.text for paragraph in doc.paragraphs])
    return text

Example:

In [14]:
file_path = 'ISACA Risk interview responses.docx'
text_from_docx = docx2txt(file_path)
len(text_from_docx)

5477

### Reading a .md to text

In [15]:
def md2txt(file_path):
    try:
        with open(file_path, "r", encoding="utf-8") as file:
            return file.read()
    except FileNotFoundError:
        return "Error: File not found."
    except Exception as e:
        return f"Error: {e}"

Example:

In [16]:
file_path = 'northwave_site.md'
text_from_md = md2txt(file_path)
len(text_from_md)

17307

### Converting .json into flatten list of strings

In [17]:
import json

def json2txt(file_path):  
    def _flatten_json(data, prefix=""):
        items = []
        
        if isinstance(data, dict):
            for key, value in data.items():
                new_prefix = f"{prefix}.{key}" if prefix else key
                
                if isinstance(value, (dict, list)):
                    items.extend(_flatten_json(value, new_prefix))
                else:
                    items.append(f"{new_prefix}: {str(value)}")
                    
        elif isinstance(data, list):
            for i, item in enumerate(data):
                new_prefix = f"{prefix}[{i}]"
                
                if isinstance(item, (dict, list)):
                    items.extend(_flatten_json(item, new_prefix))
                else:
                    items.append(f"{new_prefix}: {str(item)}")
                    
        return items
    
    # Read the JSON file
    try:
        with open(file_path, "r", encoding="utf-8") as file:
            json_data = json.load(file)
        return "\n".join(_flatten_json(json_data))
        
    except Exception as e:
        return [f"Error: {e}"]

Example:

In [18]:
file_path = 'text.json'
text_from_json = json2txt(file_path)
len(text_from_json)

245

In [19]:
text_from_json

'user.name: John Doe\nuser.age: 30\nuser.interests[0]: programming\nuser.interests[1]: music\nuser.interests[2]: sports\norders[0].id: 1\norders[0].product: laptop\norders[0].price: 999.99\norders[1].id: 2\norders[1].product: phone\norders[1].price: 599.99'

### Fetching ETDA data (https://apt.etda.or.th/cgi-bin/aptgroups.cgi)

In [22]:
import requests
import json

etda_actors = requests.get('https://apt.etda.or.th/cgi-bin/getmisp.cgi?o=g')
etda_actors = etda_actors.json()

with open("etda_actors.json", "w", encoding="utf-8") as file:
    json.dump(etda_actors, file, indent=4, ensure_ascii=False)

In [20]:
file_path = 'etda_actors.json'
text_from_etda_actors = json2txt(file_path)

In [23]:
import requests
import json

etda_actor_cards = requests.get('https://apt.etda.or.th/cgi-bin/getcard.cgi?g=all&o=j')
etda_actor_cards = etda_actor_cards.json()

with open("etda_actor_cards.json", "w", encoding="utf-8") as file:
    json.dump(etda_actor_cards, file, indent=4, ensure_ascii=False)

In [21]:
file_path = 'etda_actor_cards.json'
text_from_etda_actors_cards = json2txt(file_path)

### Fetching MITRE Groups (https://attack.mitre.org/groups/)

In [24]:
import requests
import json

mitre_actors = requests.get('https://raw.githubusercontent.com/mitre/cti/master/enterprise-attack/enterprise-attack.json')
mitre_actors = mitre_actors.json()

with open("mitre_actors.json", "w", encoding="utf-8") as file:
    json.dump(mitre_actors, file, indent=4, ensure_ascii=False)

In [25]:
file_path = 'mitre_actors.json'
text_from_mitre_actors = json2txt(file_path)

## Creating the text embeddings using a pre-trained transformer model. I.e. text to token to vector!

Check: https://huggingface.co/sentence-transformers

- sentence-transformers/all-MiniLM-L6-v2
- sentence-transformers/all-mpnet-base-v2 : MPNet, which is a transformer model similar to BERT. Hidden size of 768, meaning that each token in the input gets mapped to a 768-dimensional vector. No matter how many words you feed, the final representation is always 768-dimensional because it represents the compressed meaning of the sentence in a fixed-size vector.
- distilbert-base-uncased

In [26]:
from huggingface_hub import login
login(HUGGINGFACE_TOKEN) # ATTENTION: to get a token: https://huggingface.co/settings/tokens

In [27]:
# !pip install torch ipywidgets
# !pip install -U transformers sentence-transformers

from transformers import AutoTokenizer, AutoModel
import torch
import numpy as np

model_name = "sentence-transformers/all-mpnet-base-v2"  
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModel.from_pretrained(model_name)

def text2embedding(text):
    # Tokenize input text and convert to tensor
    inputs = tokenizer(text, return_tensors="pt", truncation=True, max_length=512) #returns PyTorch tensors
    
    # Forward pass through the model to get hidden states
    with torch.no_grad():
        outputs = model(**inputs)
    
    # Get the embeddings from the last hidden layer
    hidden_states = outputs.last_hidden_state  # Shape: (1, seq_len, hidden_dim)
    
    # Average pool over the token embeddings to get a single vector
    embedding = hidden_states.mean(dim=1).squeeze().numpy()  # Shape: (hidden_dim,)
    
    return embedding

Example:

In [28]:
embedding1 = text2embedding(text_from_docx)
len(embedding1)

768

In [29]:
embedding2 = text2embedding(text_from_md)
len(embedding2)

768

In [30]:
embedding3 = text2embedding(text_from_pdf)
len(embedding3)

768

In [31]:
embedding4 = text2embedding(text_from_json)
len(embedding4)

768

In [32]:
embedding5 = text2embedding(text_from_etda_actors)
len(embedding5)

768

In [33]:
embedding6 = text2embedding(text_from_etda_actors_cards)
len(embedding6)

768

In [34]:
embedding7 = text2embedding(text_from_mitre_actors)
len(embedding7)

768

## The vector database (for similarity search)

1. FAISS (Facebook AI Similarity Search)
2. Weaviate
3. Pinecone
4. ChromaDB
5. Milvus
6. Qdrant
7. LanceDB (anythingLLM)
8. Zilliz Cloud
9. Arrant 
10. AstraDB

In [35]:
# !pip install faiss-cpu
import faiss
dimension = 768 #check the dimension of your embeddings
index = faiss.IndexFlatL2(dimension) #Types of index: IndexFlatL2, IndexIVFFlat, IndexHNSWFlat

### Loading files from a folder into the database (FAISS)

In [39]:
import os

def load_and_index_files(folder_path, index):
    documents = []
    for filename in os.listdir(folder_path):
        file_path = os.path.join(folder_path, filename)
        
        # Check file extension and load text
        if filename.endswith(".pdf"):
            text = pdf2txt(file_path)
        elif filename.endswith(".docx"):
            text = docx2txt(file_path)
        elif filename.endswith(".json"):
            text = json2txt(file_path)
        elif filename.endswith(".md"):
            text = md2txt(file_path)
        else:
            continue  # Skip non-supported files
        
        # Embed and add to index
        embedding = text2embedding(text)
        index.add(np.array([embedding]))
        
        # Track document content and metadata (optional)
        documents.append({"filename": filename, "content": text, "embedding": embedding})
    
    return documents

Example:

In [40]:
folder_path = '.'
documents = load_and_index_files(folder_path, index)
print(f"Total documents indexed: {len(documents)}")

Total documents indexed: 7


### Function to search for documents similar (top_k elements, KNN) to the query/user_prompt converted to embedding 

In [41]:
def retrieve_documents(query_embedding, index, documents, top_k=3):
    # Search for the nearest neighbors
    distances, indices = index.search(np.array([query_embedding], dtype='float32'), top_k)
    
    # Print indices for debugging
    # print(f"indices: {indices}, distances: {distances}")
    
    # Ensure indices is structured correctly and has expected dimensions
    if indices.size == 0 or indices[0][0] == -1:
        print("No matching documents found.")
        return []  # Return empty list if no results are found

    print("Match:",[documents[i]['filename'] for i in indices[0] if i < len(documents)])
    # Fetch top_k documents using indices
    return [documents[i] for i in indices[0] if i < len(documents)]

# Create the RAG Workflow

In [44]:
def rag_pipeline(query, model="llama3.2", top_k=1):
    query_embedding = text2embedding(query) #Convert the query/prompt into an embedding (vector)

    relevant_docs = retrieve_documents(query_embedding, index, documents, top_k)
    # print("Relevant docs:",relevant_docs)
    
    # 3. Construct the prompt with file information
    context = "\n".join([f"File: {doc['filename']}\nContent:\n{doc['content']}" for doc in relevant_docs])
    # print("Context:",context)
    
    system_prompt = """Given the following context from relevant documents, and a follow up question, reply with an answer to the current question the user is asking. 
    Return only your response to the question given the above information following the users instructions as needed.
    Answer strictly based on the provided context. 
    If the context does not include any information relevant to the question, respond exactly with "The context does not provide information on this topic."""
    
    user_prompt = f"""Context (from relevant documents):{context}   
    
    Question: {query} 
    """
    # print("User prompt:",user_prompt)

    # 4. Call the language model
    response = ollama(model,system_prompt, user_prompt)
    return response

# ==============================================================================
# ==============================================================================
# Testing everything

In [45]:
query = "What do you know about 'Zombie Spider' from the ETDA document?"
response = rag_pipeline(query)
print(response)

Match: ['etda_actors.json']
From the provided text, I can gather the following information about "Zombie Spider":

1. **Identity**: Zombie Spider is believed to be a pseudonym for Peter Yuryevich LEVASHOV, a Russian national who was involved in operating malware and botnets.
2. **Background**: LEVASHOV was arrested in Spain in April 2017 when the final version of Kelihos was taken over. He recently pleaded guilty to operating the botnet for criminal purposes.
3. **Malware distribution**: Zombie Spider (LEVASHOV) has been linked to distributing various malware, including:
	* TrickBot
	* Zeus Panda ('Bamboo Spider')
4. **Criminal activities**: In the past, LEVASHOV was involved in other malicious activities such as:
	* Pump-and-dump stock scams
	* Date ruses
	* Credential phishing
	* Money mule recruitment
	* Rogue online pharmacy advertisements

5. **Botnet takeovers**: Zombie Spider has been associated with taking over various botnets, including the Kelihos botnet.

6. **Takedown**: In

In [46]:
query = "What are the list of url references on 'Zombie Spider' from the ETDA document?"
response = rag_pipeline(query)
print(response)

Match: ['northwave_site.md']
Based on the provided content, here is a list of URL references related to "Zombie Spider" from the ETDA (European Union's Network and Information Security Directive) document:

1. https://northwave-cybersecurity.com/ (General website URL)
2. https://northwave-cybersecurity.com/responsible-disclosure (Responsible Disclosure page)
3. https://northwave-cybersecurity.com/privacy-cookie-statement (Privacy Statement)
4. https://northwave-cybersecurity.com/cookie-statement (Cookie Statement)

Note that the "Zombie Spider" is likely a reference to the "Zombie Spider Attack" mentioned in the ETDA document, which is a type of cyber attack. However, without more specific information about the content of the ETDA document, it's difficult to provide a more detailed list of URL references related to this topic.

Please let me know if you'd like me to help with anything else!


# ↑wrong!

In [47]:
query = "From the etda actors document, list the references related to 'Zombie Spider'"
response = rag_pipeline(query)
print(response)

Match: ['etda_actors.json']
According to the etda actors document, the reference related to 'Zombie Spider' is:

https://apt.etda.or.th/cgi-bin/showcard.cgi?u=2c1d1677-f2d9-44e1-ac9a-4f7f4047e2d5


In [48]:
query = "From the etda actors document, list the refs related to 'Zombie Spider' actor"
response = rag_pipeline(query)
print(response)

Match: ['etda_actors.json']
Based on the provided document from ETDa Actors, the references (refs) related to the "Zombie Spider" actor are:

* https://apt.etda.or.th/cgi-bin/showcard.cgi?u=2c1d1677-f2d9-44e1-ac9a-4f7f4047e2d5
* https://www.crowdstrike.com/blog/inside-the-takedown-of-zombie-spider-and-the-kelihos-botnet/
* https://www.crowdstrike.com/blog/farewell-to-kelihos-and-zombie-spider/

These references are directly related to the "Zombie Spider" actor, specifically mentioning Kelihos and the takedown of the botnet.


# ↑not complete!

In [49]:
query = "From the Northwave site document, tell me what is northwave? and where Northwave is located?"
response = rag_pipeline(query)
print(response)

Match: ['northwave_site.md']
According to the provided website content, Northwave appears to be a cybersecurity company or consulting firm. The text does not explicitly state that it is a specific product or service, but rather a company that offers various cybersecurity-related services.

As for its location, according to the contact information provided on the website, Northwave is located in:

* Netherlands: Van Deventerlaan 31-51, 3528 AG Utrecht
* Postal address: PO 1305, 3430 BH, Nieuwegein

Note that there is no explicit mention of Northwave being a specific product or service, but rather an organization that offers various cybersecurity-related services.


In [50]:
query = "From the Northwave site document, could you get the address and the contact phone of Northwave? "
response = rag_pipeline(query)
print(response)

Match: ['northwave_site.md']
Based on the provided website content, I found that:

* The physical address is:
Van Deventerlaan 31-51, 3528 AG Utrecht
* The postal address is:
PO 1305, 3430 BH, Nieuwegein

Unfortunately, I could not find the contact phone number on the website.


# ↑Wrong!

In [51]:
query = "From the Northwave site document, could you list northwave's services?"
response = rag_pipeline(query)
print(response)

Match: ['northwave_site.md']
Based on the provided Northwave website content, here are their listed services:

**Business**

1. Managed Security & Privacy Office
2. State of Security Assessment
3. Data Protection Impact Assessment
4. Security Roadmap
5. Assess & Control
6. ISO 27001 FastTrack

**Bytes**

1. Managed Detection & Response
2. Rapid Response
3. Red Teaming
4. Advanced Red Teaming (ART)
5. Penetration Testing (Pentest)
6. Vulnerability Management

**Behaviour**

1. Managed Cyber Behaviour
2. Cyber Resilience Service
